In [1]:
# Define model
def DefineModel(Model):
    Model.Allocation = pyo.Var(Model.Candidate, Model.GridWords, within = pyo.Binary, initialize = 0)   # Allocate candidate words to grid

    def rule_PosOnce(Model, g):   # Allocate exactly one candidate to each grid word position
        return sum(Model.Allocation[c, g] for c in Model.Candidate) == 1
    Model.EachPositionOnce = pyo.Constraint(Model.GridWords, rule = rule_PosOnce)
    
    def rule_WordOnce(Model, c):   # Allocate each word to a grid position at most once. Optional constraint
        return sum(Model.Allocation[c, g] for g in Model.GridWords) <= 1
    Model.EachWordOnce = pyo.Constraint(Model.Candidate, rule = rule_WordOnce)

    def rule_Fit(Model, g):   # Ensure word exactly fills its allocated grid space
        return sum(Model.Allocation[c, g] * Model.Length[c] for c in Model.Candidate) == Model.GridLengths[g]
    Model.WordsFit = pyo.Constraint(Model.GridWords, rule = rule_Fit)

    def rule_Intersection(Model, g1, g2, w, h):   # The intersection of grid words must have the same letter
        if pyo.value(Model.AcrossRef[w, h]) != 0 and pyo.value(Model.DownRef[w, h]) != 0 and g1 == pyo.value(Model.AcrossRef[w, h]) - 1 \
                and g2 == pyo.value(Model.DownRef[w, h]) - 1:
            return sum(Model.Allocation[c, g1] * Model.Word[c, pyo.value(Model.AcrossPos[w, h]) - 1] for c in Model.Candidate) == \
                sum(Model.Allocation[c, g2] * Model.Word[c, pyo.value(Model.DownPos[w, h]) - 1] for c in Model.Candidate)
        else:
            return pyo.Constraint.Skip
    Model.Crossover = pyo.Constraint(Model.GridWords, Model.GridWords, Model.GridWidth, Model.GridHeight, rule = rule_Intersection)

    def rule_Obj(Model):
        return sum(sum(Model.Allocation[c, g] for g in Model.GridWords) * Model.Frequency[c] for c in Model.Candidate)
    Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.maximize)